In [11]:
import wandb
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
import tqdm

import torch
from torch.utils.data import Dataset, DataLoader
import seaborn as sns

In [3]:
pathways_df = pd.read_csv('data/pathways.csv')
pathways_df.set_index("Object ID", inplace=True)
pathways_df["Reaction-List"] = pathways_df["Reaction-List"].str.split(" // ")
pathways_df["Species"] = pathways_df["Species"].str.split(" // ")
pathways_df["In-Pathway"] = pathways_df["In-Pathway"].str.split(" // ")
pathways_df["Super-Pathways"] = pathways_df["Super-Pathways"].str.split(" // ")

reactions_df = pd.read_csv('data/reactions.csv')
reactions_df["Left"] = reactions_df["Left"].str.split(" // ")
reactions_df["Right"] = reactions_df["Right"].str.split(" // ")
reactions_df["In-Pathway"] = reactions_df["In-Pathway"].str.split(" // ")
reactions_df["Substrates"] = reactions_df["Substrates"].str.split(" // ")
reactions_df["Enzymatic-Reaction"] = reactions_df["Enzymatic-Reaction"].str.split(" // ")
reactions_df.set_index("Object ID", inplace=True)

compounds_df = pd.read_csv("data/compounds.csv")
compounds_df.set_index("Object ID", inplace=True)

species = {}
for i, row in pathways_df.iterrows():
    if type(row["Species"]) == list:
        for s in row["Species"]:
            if s in species:
                species[s].append(i)
            else:
                species[s] = [i]

In [4]:
species

{'TAX-38868': ['PWY-6990', 'PWY-5813', 'PWY-7680'],
 'TAX-3888': ['PWY-2467',
  'PWY-2464',
  'PWY-7101',
  'PWY-6147',
  'PWY-6333',
  'PWY-5342',
  'PWY-7033',
  'PWY-6963',
  'PWY-6964',
  'PWY-2261',
  'PWY-3941',
  'PWY-3561',
  'ASPSYNII-PWY',
  'TRIGLSYN-PWY',
  'PWY-2501',
  'PWY-6799',
  'PWY-4381',
  'PWY-3841',
  'PWY-5034',
  'PWY-5070',
  'PWY-5035',
  'PWY-102',
  'GLUCONEO-PWY',
  'GLUT-REDOX-PWY',
  'PWY-4081',
  'GLYCLEAV-PWY',
  'PWY-1042',
  'PWY-1061',
  'PWY-5348',
  'ASPARAGINE-DEG1-PWY',
  'PWY-4002',
  'PWY0-1264',
  'PWY4FS-7',
  'PWY4FS-8',
  'PWY-2',
  'PWY-7199',
  'PWY-7184',
  'PYRUVDEHYD-PWY',
  'PWY-7173',
  'PWY-6932',
  'PWY-6936',
  'PWY-5337',
  'PWY-622',
  'PWY-7238',
  'PWY-3801',
  'PWY-3282',
  'PWY-7345',
  'PWY-4762',
  'PWY-5464',
  'PWY-5156',
  'PWY-6266',
  'PWY-5052',
  'PWY-5053',
  'PHOSLIPSYN2-PWY',
  'PWY-2229',
  'PWY-6305',
  'PWY-7211',
  'PWY-6395',
  'PWY-6612',
  'PWY-6614',
  'PWY-5110',
  'PWY-4',
  'PWY-4841',
  'PWY-82',
  '

In [5]:
class SimpleTripletDataset(Dataset):
    def __init__(self):
        self.organism_pathways = species
        self.organisms = list(species.keys())
        self.pathways = set(pathways_df.index)
        
    def __len__(self):
        return len(self.organisms)
    
    def __getitem__(self, idx):
        org = self.organisms[idx]
        org_pathways = self.organism_pathways[org]
    
        pos_pathway = random.choice(org_pathways)
        neg_pathway = random.choice(list(self.pathways - set(org_pathways)))
        
        return org, pos_pathway, neg_pathway
        
        
dataset = SimpleTripletDataset()
loader = DataLoader(dataset, batch_size=4, shuffle=True)

for org, pos, neg in loader:
    print("Organism:", org)
    print("Positive Pathway:", pos)
    print("Negative Pathway:", neg)
    break
    # Break here just to avoid long output; remove it in actual usage

Organism: ('TAX-876169', 'ORG-6201', 'TAX-169679', 'TAX-3983')
Positive Pathway: ('PWY-8038', 'PWY-6996', 'PWY-6590', 'PWY-3121')
Negative Pathway: ('PWY-8159', 'PWY-5788', 'PWY-7929', 'PWY-5203')


In [6]:
def make_pathway_graph(
        pathway_id: str,
):
    try:
        pathway = pathways_df.loc[pathway_id]
    except KeyError:
        return None, [], []
    
    try:
        reactions = reactions_df.loc[pathway["Reaction-List"]]
    except KeyError:
        return None, [], []
    #reactions = reactions_df.loc[pathway["Reaction-List"]]
        
    edges, compound_v, reaction_v = set(), set(), set()
    for i, reaction in reactions.iterrows():
        reaction_v.add(i)
        for l in reaction["Left"]:
            compound_v.add(l)
            edges.add((l, i))
        for r in reaction["Right"]:
            compound_v.add(r)
            edges.add((i, r))
    
    vertices = list(compound_v) + list(reaction_v)
    adj_matrix = np.zeros((len(vertices), len(vertices)))
    for edge in edges:
        adj_matrix[vertices.index(edge[0]), vertices.index(edge[1])] = 1
        
    return adj_matrix, list(reaction_v), list(compound_v)

substrates_count = {}

for substrates in reactions_df["Substrates"]:
    for substrate in substrates:
        if substrate in substrates_count:
            substrates_count[substrate] += 1
        else:
            substrates_count[substrate] = 1
            
compound_popularity = pd.Series(substrates_count, name="Count").sort_values(ascending=False)

def remove_compounds_from_graph(
        adj_matrix, reactions, compounds, to_remove
):
    all_labels = compounds + reactions
    indices_to_keep = [i for (i, x) in enumerate(all_labels) if x not in to_remove]
    
    adj_matrix = adj_matrix[indices_to_keep, :][:, indices_to_keep]
    reactions = [r for r in reactions if r not in to_remove]
    compounds = [c for c in compounds if c not in to_remove]
    
    return adj_matrix, reactions, compounds

from skfp.fingerprints import AtomPairFingerprint

# TOTAL BASELINE 
def embed_graph(compounds):
    compounds = [compounds_df.loc[c, "SMILES"] for c in compounds if c in compounds_df.index]
    
    try:
        fp = AtomPairFingerprint(n_jobs=-1)
        result = fp.transform(compounds).sum(axis=0)
    except:
        return np.zeros(2048)
    
    return result



In [8]:
cached_base_embeddings = {}

for pwy in tqdm.tqdm(pathways_df.index):
    graph = make_pathway_graph(pwy)
    if graph is None:
        continue
    adj_matrix, reactions, compounds = graph
    embedding = embed_graph(compounds)
    cached_base_embeddings[pwy] = torch.tensor(embedding.astype(np.float32))

  0%|          | 1/3538 [00:01<1:42:24,  1.74s/it][17:39:54] WARNING: not removing hydrogen atom without neighbors
[17:39:54] WARNING: not removing hydrogen atom without neighbors
[17:39:54] WARNING: not removing hydrogen atom without neighbors
[17:39:54] WARNING: not removing hydrogen atom without neighbors
[17:39:54] WARNING: not removing hydrogen atom without neighbors
[17:39:54] WARNING: not removing hydrogen atom without neighbors
  0%|          | 8/3538 [00:01<10:05,  5.83it/s]  [17:39:54] WARNING: not removing hydrogen atom without neighbors
[17:39:54] WARNING: not removing hydrogen atom without neighbors
[17:39:54] WARNING: not removing hydrogen atom without neighbors
[17:39:54] WARNING: not removing hydrogen atom without neighbors
[17:39:54] WARNING: not removing hydrogen atom without neighbors
  0%|          | 16/3538 [00:01<04:33, 12.89it/s][17:39:54] WARNING: not removing hydrogen atom without neighbors
[17:39:54] WARNING: not removing hydrogen atom without neighbors
[17:39

In [9]:
class SimplePathwayEncoder(torch.nn.Module):
    def __init__(self, embedding_dim=128):
        super(SimplePathwayEncoder, self).__init__()
        self.fc1 = torch.nn.Linear(2048, embedding_dim)
            
    def forward(self, pathways):
        embeddings = []
        for p in pathways:
            if p in cached_base_embeddings:
                embeddings.append(cached_base_embeddings[p])
                continue
            graph = make_pathway_graph(p)
            adj_matrix, reactions, compounds = graph
            embedding = embed_graph(compounds)
            embeddings.append(torch.tensor(embedding.astype(np.float32)))
        x = torch.stack(embeddings)
        x = self.fc1(x)
        return x

In [14]:
wandb.init(project='Metacyc', config={
    "learning_rate": 0.001,
    "epochs": 1000,
    "batch_size": 16,
    "embedding_dim": 256
})

config = wandb.config

dataset = SimpleTripletDataset()
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

EMBEDDING_DIM = config.embedding_dim
organism_embedding = torch.nn.Embedding(len(species), EMBEDDING_DIM)
pathway_encoder = SimplePathwayEncoder(EMBEDDING_DIM)
optimizer = torch.optim.Adam(
    list(organism_embedding.parameters()) + list(pathway_encoder.parameters()), 
    lr=0.001
)

criterion = torch.nn.TripletMarginLoss(margin=1.0, p=2)


for epoch in tqdm.tqdm(range(config.epochs)):
    total_loss = 0
    num_batches = 0
    print(f"Epoch {epoch+1}")

    for organisms, pos_pathways, neg_pathways in dataloader:
        optimizer.zero_grad()

        organism_idx = torch.tensor([list(species.keys()).index(o) for o in organisms])
        org_vectors = organism_embedding(organism_idx)
        
        pos_path_vectors = pathway_encoder(pos_pathways)
        neg_path_vectors = pathway_encoder(neg_pathways)
        
        loss = criterion(org_vectors, pos_path_vectors, neg_path_vectors)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        num_batches += 1

        # Optionally log batch-level metrics
        wandb.log({"Batch Loss": loss.item()})

    average_loss = total_loss / num_batches
    print(f"Average Loss for Epoch {epoch+1}: {average_loss:.4f}")
    
    # Log epoch-level metrics
    wandb.log({"Epoch": epoch + 1, "Average Loss": average_loss})

wandb.finish()

  0%|          | 0/1000 [00:00<?, ?it/s]

Epoch 1


  0%|          | 1/1000 [00:05<1:25:30,  5.14s/it]

Average Loss for Epoch 1: 8.2028
Epoch 2


  0%|          | 2/1000 [00:10<1:28:07,  5.30s/it]

...

100%|█████████▉| 999/1000 [1:15:08<00:04,  4.33s/it]

Average Loss for Epoch 999: 2.5555
Epoch 1000


100%|██████████| 1000/1000 [1:15:12<00:00,  4.51s/it]

Average Loss for Epoch 1000: 3.0665


Average Loss,█▅▆▆▅▄▅▅▅▅▄▄▄▃▄▃▄▃▄▃▃▃▄▃▃▃▂▃▃▂▃▃▂▂▃▃▂▂▂▁
Batch Loss,▂▅▄▅▁▁▃▂▁▂▁▁▁▂▃▁▁▁▁█▃▁▁▁▁▂▁▂▁▂▄▁▁▃▁▁▁▂▁▁
Epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
Average Loss,3.06652
Batch Loss,19.50455
Epoch,1000
